In [13]:
import numpy as np
#import pandas as pd
import os
import transformers
from tqdm import tqdm
import scipy

import pretty_midi
import IPython.display
import re
import pickle

import subprocess


from transformers import GPT2Tokenizer, GPT2LMHeadModel#, GPT2Config
#from transformers import TextDataset, DataCollatorForLanguageModeling
#from transformers import Trainer, TrainingArguments

In [14]:
FS = 44100
MIDI_TEXT_FILENAME = './inputs/midi_text_data.txt'
MODEL_OUT_DIR = './modeloutput/'

In [7]:
def generate_music(model_path, prompt="Bach", max_length=512, num_return_sequences=1, temperature=1.0):
    """
    Generate music using your fine-tuned GPT-2 model
    
    Args:
        model_path: Path to the fine-tuned model
        prompt: Text prompt to start generation (e.g., "Composer: Bach")
        max_length: Maximum length of generated sequence
        num_return_sequences: Number of different sequences to generate
        temperature: Controls randomness (lower = more deterministic)
        
    Returns:
        List of generated MIDI objects
    """
    # Load model and tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    model = GPT2LMHeadModel.from_pretrained(model_path)
    
    # Format the prompt with the right structure
    formatted_prompt = f"<|startofpiece|>\nX: 1\n{prompt}\n"
    
    # Encode the prompt
    input_ids = tokenizer.encode(formatted_prompt, return_tensors="pt")
    
    # Generate continuation
    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=50,
        top_p=0.95,
        num_return_sequences=num_return_sequences,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
    )
    
    # Decode and parse the output
    generated_sequences = []
    for sequence in output_sequences:
        text = tokenizer.decode(sequence, skip_special_tokens=False)
        
        # Make sure we stop at the end of piece token if present
        if "<|endofpiece|>" in text:
            text = text.split("<|endofpiece|>")[0] + "<|endofpiece|>"
            
        generated_sequences.append(text)
        
    # Convert text to MIDI
    midi_objects = []# [text_to_midi(seq) for seq in generated_sequences]
    
    return midi_objects, generated_sequences

midi_objects, generated_sequences = generate_music('./modeloutput/', prompt="T: debussy")

In [19]:
def text_to_midi(text, debug = False):
    """Convert generated text back to MIDI format"""
    # Initialize a PrettyMIDI object
    midi = pretty_midi.PrettyMIDI()
    
    # Parse the text to extract notes
    lines = text.strip().split('\n')
    #print(lines)
    
    # Group notes by instrument
    instruments = {}
    
    # Extract all notes
    for line in lines:
        # Skip metadata and special tokens
        if line.startswith('<|') or not ('start_time:' in line and 'note_value:' in line):
            if debug:
                print(line, "skipped") 
            continue
            
        # Parse note information using regex
        start_match = re.search(r'start_time: ([0-9.]+)', line)
        end_match = re.search(r'end_time: ([0-9.]+)', line)
        note_match = re.search(r'note_value: (\d+)', line)
        vel_match = re.search(r'velocity: (\d+)', line)
        inst_match = re.search(r'instrument_program_number: (\d+)', line)
        
        if all([start_match, end_match, note_match, vel_match, inst_match]):
            start_time = float(start_match.group(1))
            end_time = float(end_match.group(1))
            pitch = int(note_match.group(1))
            velocity = int(vel_match.group(1))
            program = int(inst_match.group(1))

            if debug:
                print("got:", start_time, end_time, pitch, velocity, program)
            
            # Keep notes in valid MIDI range
            if 0 <= pitch <= 127 and velocity > 0:
                # Add instrument if needed
                if program not in instruments:
                    instruments[program] = pretty_midi.Instrument(program=program)
                    
                # Create a note
                note = pretty_midi.Note(
                    velocity=min(velocity, 127),  # Ensure valid velocity
                    pitch=pitch,
                    start=max(0, start_time),  # Ensure valid timing
                    end=max(start_time + 0.1, end_time)  # Ensure note has duration
                )
                
                instruments[program].notes.append(note)
    
    # Add instruments to the MIDI file
    for instrument in instruments.values():
        midi.instruments.append(instrument)
        
    return midi

In [23]:
midi_objects, generated_sequences = generate_music('./modeloutput/', 'Beethoven')
print(len(generated_sequences))
test_midi = text_to_midi(generated_sequences[0], debug = True)
test_midi = test_midi.synthesize()
IPython.display.Audio(test_midi, rate=44100)


# Pickling (serialization)
#with open('data.pkl', 'wb') as file:
#    pickle.dump(test_midi, file)

# Unpickling (deserialization)
#with open('data.pkl', 'rb') as file:
#    loaded_data = pickle.load(file)
#IPython.display.Audio(loaded_data, rate=44100)

1
<|startofpiece|>  skipped
Composer: Beethoven skipped
got: 0.0 0.18359375 58 80 0
got: 0.0 0.18359375 55 80 0
got: 0.1875 0.37109375 57 80 0
got: 0.1875 0.37109375 53 80 0
got: 0.375 0.55859375 58 80 0
got: 0.375 0.55859375 55 80 0
got: 0.5625 0.7460938 57 80 0
got: 0.5625 0.7460938 53 80 0
got: 0.75 0.859375 60 80 0
got: 0.75 0.96875 56 80 0
got: 0.75 0.96875 52 80 0
got: 0.75 0.96875 53 80 0
got: 1.5 1.734375 62 80 0
got: 1.5 1.96875 50 80 0
got: 1.75 1.734375 53 80 0
start_time: 1.75 end_time: 1.875 note_time: 2 note_value skipped


In [25]:
scipy.io.wavfile.write('beethovenshort.wav', FS, test_midi)

In [15]:
def abcToMidi(abc_path, midi_path): # for testing
    try:
        subprocess.run(["abc2midi", abc_path, "-o", midi_path], check=True)
        #print(f"[OK] Saved MIDI to {midi_path}")
    except subprocess.CalledProcessError as e:
        print(f"[ERROR] ABC to MIDI conversion failed: {e}")

In [21]:
test_filename = "fartingsomuch.abc"
midi_filename = "fartingsomuch.mid"
wav_filename = "fartingsomuch.wav"
with open(test_filename, "w") as fart:
    fart.writelines(generated_sequences)
abcToMidi(test_filename, midi_filename)
test_midi = pretty_midi.PrettyMIDI(midi_filename)
test_synth = test_midi.synthesize()
scipy.io.wavfile.write(wav_filename, FS, test_synth.astype(np.int16))
IPython.display.Audio(test_synth, rate=44100)

4.88 February 22 2024 abc2midi
writing MIDI file fartingsomuch.mid
Warning in line-char 13-163 : Track 1 Bar 2 has 23/4 time units while the time signature has 4


/home/paul-dutton/miniconda3/lib/python3.12/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
